In [3]:
!pip3 install beautifulsoup4

In [211]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
import io

In [212]:
no_pages = 2

def remove_invalid_chars(string):
    alphanumeric = ""
    for character in string:
        if(character=="," or character=="." or character=="₹"):
            alphanumeric += character
        elif character.isspace():
            alphanumeric += character
        elif character.isalnum():
            alphanumeric += character
        
    return alphanumeric

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    All_Details=[]
    
    r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)

    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})
   

        if name is not None:
            valid_string = remove_invalid_chars(n[0]['alt']) 
            All_Details.append(valid_string)
        else:
            All_Details.append("Unknow Product")
    
    
        if author is not None:
            valid_string = remove_invalid_chars(author.text)
            All_Details.append(valid_string)
        elif author is None:
            sec_author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if sec_author is not None:
                valid_string = remove_invalid_chars(sec_author.text)
                All_Details.append(valid_string)
            else:
                All_Details.append("Author not specified")

   
        if rating is not None:
            valid_string = remove_invalid_chars(rating.text)
            All_Details.append(valid_string)
        else:
            All_Details.append("No ratings")

  
        if users_rated is not None:
            valid_string = remove_invalid_chars(users_rated.text)
            All_Details.append(valid_string)
        else:
            All_Details.append("0")
    
        if price is not None:
            valid_string = remove_invalid_chars(price.text)
            All_Details.append(valid_string.replace("₹","Rs."))
        else:
            All_Details.append("Price not specified")
    return All_Details
    
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
   # print(results)

In [216]:
filename = "products.csv"
with io.open(filename,"w",encoding="utf-8") as f:
    headers = "Book_Name, Author, Rating, Customers_Rated, Price\n"
#f.write(headers)
    for item in results:
        for i in item:
            f.write(i+"\n")